Data cleaning is arguably the most important step in most data science projects. Without cleaning many data sets will often work poorly when it comes to modelling, that's if they work at all. A thorough cleaning process avoids any challenges later down the road when your focus should be on getting the most from your data.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('DataAnalyst.csv')
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\r\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\r\n\r\nProvides analytical and techni...,3.8,Visiting Nurse Service of New York\r\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\r\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,-1
3,3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\r\nRemote:Yes\r\n...,4.1,Celerity\r\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,-1
4,4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\r\n\r\nFanDuel Group is a ...,3.9,FanDuel\r\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,True


At first glance, we can see that many of the columns need some kind of cleaning. I've listed below the following changes to be made:

|Field|Action|
|:---|:---|
|Job Title|Need to separate the job title|
|Salary Estimate|Value range needs parsing and converting to an average|
|Job Description|Parse skill data (python, scikit, etc.)|
|Company Name|Remove additional rating text|
|Location|Parse the state ID only|
|Founded|Convert this to the company age|

Let's get started!

In [3]:
#job title parsing
df['job_title'] = df['Job Title'].apply(lambda x: x.split(',')[0])

In [4]:
#salary parsing
df = df[df['Salary Estimate'] != '-1']

salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])

remove_symbols = salary.apply(lambda x: x.replace('K', '').replace('$',''))

df['min_salary'] = remove_symbols.apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = remove_symbols.apply(lambda x: int(x.split('-')[1]))
df['avg_salary'] = (df.min_salary + df.max_salary) / 2

In [5]:
#job description parsing
# use count vectoriser on the whole data set to get the most popular words
# then make these words in columns i.e. python_y/n, sql_y/n and check for those

df['python_yn'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

In [6]:
#company name parsing
df['company_text'] = df.apply(lambda x: x['Company Name'] if x['Rating'] < 0 else x['Company Name'][:-5], axis = 1)

In [7]:
#location
df['location_state'] = df['Location'].apply(lambda x: x.split(',')[0])
df['work_at_hq'] = df.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis = 1)

In [8]:
#company age
df['age'] = df.Founded.apply(lambda x: x if x<1 else 2020 - x)

Now we can start dropping columns that definitely do not contain any useful information going forward. Some of the columns we keep might not actually be used in the modelling, but we'll keep them around in case we do need them in a future phase of the project.

In [9]:
to_drop = ['Unnamed: 0']

df_output = df.drop(columns = to_drop)

df_output.to_csv('job_data_cleaned.csv')